In [1]:
#IMPORTANT

#because this is a very large website, make sure to run jupyter with: 
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000 
#or else it may run out of data

In [2]:
import requests
#selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#
from bs4 import BeautifulSoup
from datetime import date
from time import sleep
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date
from datetime import datetime




In [3]:
#lists for data

brand = [] #
product_type = [] #
product_name = [] #
product_description = []#
price = []#
sales_price = []#
color = [] #reference the wearpact crawler for how to get the image of each different color
material = []#
product_url = []#
picture_url = []# 
timestamp = []#
gender = [] #looks like the site is all womens clothing
new_arrival = []#
new_arrival_names=[] 
# you will definitely need to use scrolling loop i used 
#maybe add a sleep somewhere in there to give it extra time to go down
#as it seems you have to fully hit the bottom before it puts new products

current_date = date.today()

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #selenium
driver.set_window_size(1120, 1000)

#to get a webpage with selenium you use driver.get(url), and if you want to crawl what the driver sees using bs4 you use
    # soup = BeautifulSoup(driver.page_source,'html.parser'), keep in mind you can also find and access attributes of elements
    # with selenium using BY. I used this a few times in mine (needed for clicking)


"""
#Useful example of having the driver scroll down the page slowly to generate any new products if the page gets larger as you scroll
total_height = int(driver.execute_script("return document.body.scrollHeight"))
    i = 1
    while i < total_height:
        i+=5
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        driver.execute_script("window.scrollTo(0, {});".format(i))
"""



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\813ro\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


'\n#Useful example of having the driver scroll down the page slowly to generate any new products if the page gets larger as you scroll\ntotal_height = int(driver.execute_script("return document.body.scrollHeight"))\n    i = 1\n    while i < total_height:\n        i+=5\n        total_height = int(driver.execute_script("return document.body.scrollHeight"))\n        driver.execute_script("window.scrollTo(0, {});".format(i))\n'

In [4]:
#getting info from product

#everything is generated through JS on the page, therefore you cannot use beautiful soup (selenium is needed)
def getProductInfo(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    
    #brand (always pact)
    prod_brand = "TheReformation"
    
    #name
    prod_title_div = soup.find('h1', {'class':'pdp__name'})
    if prod_title_div == None:
        return None
    prod_title = prod_title_div.get_text()
    
    
    #product_type + gender
    prod_ol = soup.find('ol', {'class':'breadcrumbs'})
    
    '''
    prod_gender = prod_ol.contents[0].get_text()
    if prod_gender == "baby":
        prod_gender = "kids"
    '''
    prod_gender = "womens" #always womens clothing
    
    prod_type = prod_ol.contents[1].a.get_text()
    print(prod_type)
    #prices
    prod_price = None
    prod_sale_price = soup.find('span',{'class':'price--formatted'})
    '''
    if prod_sale:
        prod_bundle = prod_sale.get_text()
        price_split = prod_bundle.split('$')
        prod_price = '$' + price_split[2]
        prod_sale_price = '$' + price_split[1]
    else:
        prod_sale_price = soup.find('div', {'class':'dollar'}).get_text()
    '''
    
    #desc
    desc_sentence = soup.find("div", {'class':'cms-generic-copy'})
    print(desc_sentence.get_text())
    '''
    desc_ul = desc_div.contents[0]
    desc_sentence = ''
    for child in desc_ul.children:
        desc_sentence += child.get_text() + ". "
    '''
    #Material (see explanation in previous cell)
    prod_material = soup.find("div", {'class':'margin-b--15'})
    
    
    #New Arrival
    is_new = prod_title in new_arrival_names
    
    #color + image url are unique, as they are a different entry but share almost all of the same data (including url)
        #therefore this really makes as many entries as there are color options (using clicks from selenium)
    #color_list = soup.find('div',{'class':'product-attributes__contents'})
    color_list = soup.find_all('div',{'class':'flex-flow-wrap'})[3]
    
    count = 1
    for child in color_list.find_all('button'):
        #color name
        
        prod_color = child.get('title') #color name
        
        #image url (click with selenium)
        color_btn = driver.find_element(By.XPATH, f"//div[@data-attr-group='color']/div[2]/button[{count}]")
        
        color_btn.click()
        sleep(1)
        image_tag = driver.find_element(By.XPATH, "//img[@itemprop='image']")
        prod_image_url = image_tag.get_attribute("src")
        sleep(1)
        
        count += 1
        #creating df entry
        timestamp.append(current_date)
        product_url.append(url)
        brand.append(prod_brand)
        product_name.append(prod_title)
        gender.append(prod_gender)
        product_type.append(prod_type)
        price.append(prod_price)
        sales_price.append(prod_sale_price)
        product_description.append(desc_sentence)
        material.append(prod_material)
        new_arrival.append(is_new)
        color.append(prod_color)
        picture_url.append(prod_image_url)
    
#test
#getProductInfo("https://www.thereformation.com/products/mylie-two-piece/1309636PAG.html?dwvar_1309636PAG_color=JUT&quantity=1")

In [5]:

'''driver.set_page_load_timeout(100)
driver.set_script_timeout(100)
#new arrivals
#this can be done 

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scrolls to bottom of page
driver.get("https://www.thereformation.com/new")
    
total_height = int(driver.execute_script("return document.body.scrollHeight"))
i = 1
while i < total_height:
    i+=10
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    driver.execute_script("window.scrollTo(0, {});".format(i))

    
sleep(0.3)

soup = BeautifulSoup(driver.page_source,'html.parser')
prod_list = list(soup.find_all('div',{'class':'product-tile__name'}))

for i in prod_list:
    new_arrival_names.append(i.get_text())
    

 '''

'driver.set_page_load_timeout(100)\ndriver.set_script_timeout(100)\n#new arrivals\n#this can be done \n\ndriver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scrolls to bottom of page\ndriver.get("https://www.thereformation.com/new")\n    \ntotal_height = int(driver.execute_script("return document.body.scrollHeight"))\ni = 1\nwhile i < total_height:\n    i+=10\n    total_height = int(driver.execute_script("return document.body.scrollHeight"))\n    driver.execute_script("window.scrollTo(0, {});".format(i))\n\n    \nsleep(0.3)\n\nsoup = BeautifulSoup(driver.page_source,\'html.parser\')\nprod_list = list(soup.find_all(\'div\',{\'class\':\'product-tile__name\'}))\n\nfor i in prod_list:\n    new_arrival_names.append(i.get_text())\n    \n\n '

In [6]:
#getting links to crawl

product_links = []

driver.set_page_load_timeout(150)
driver.set_script_timeout(150)

#new arrivals
#this can be done 


driver.get("https://www.thereformation.com/clothing?page=1")

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scrolls to bottom of page

total_height = int(driver.execute_script("return document.body.scrollHeight"))
i = 1
while i < total_height:
    i+=10
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    driver.execute_script("window.scrollTo(0, {});".format(i))

    
sleep(0.3)

soup = BeautifulSoup(driver.page_source,'html.parser')
sleep(1)
#prod_list = list(soup.find('div',{'itemtype':'http://schema.org/SomeProducts%27%7D'}))
prod_list = soup.find('div', {'data-search-component','product-grid'})

for child in prod_list.children:
    print(child)
    prod_link = child.find('a',{'itemprop':'url'})
    if prod_link:
            product_links.append("https://thereformation.com" + prod_link['href'])
            print("https://thereformation.com" + prod_link['href'])
    else:
        print("FAIL")

AttributeError: 'NoneType' object has no attribute 'children'

In [ ]:
#all
for link in product_links:
    print(link)
    getProductInfo(link)

In [ ]:
#creating dataframe and CSV
dataframe = pd.DataFrame(list(zip(brand, product_type, product_name, product_description, price, sales_price, color, material, product_url, picture_url, timestamp, gender, new_arrival)), columns = ["Brand", "Product Type", "Product Name", "Description", "Price", "Sales Price", "Color", "Material", "Product URL", "Picture URL", "Timestamp", "Gender", "New Arrival"])
dataframe.head()
#dataframe.to_csv('reformation.csv')